In [1]:
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px

# 1. Obtention des données
## Chargement des fichiers
On charge ici tous les fichiers de l'export (préalablement compressés pour préserver la place sur le disque)

In [2]:
analyses_orthophosphate = pd.read_csv("./naiades_export_orthophosphate/Analyses.CSV",sep=";",parse_dates=["DatePrel"])

In [3]:
pd.set_option('display.max_columns', None)
analyses_orthophosphate.columns


Index(['CdStationMesureEauxSurface', 'LbStationMesureEauxSurface', 'CdSupport',
       'LbSupport', 'CdFractionAnalysee', 'LbFractionAnalysee',
       'CdPrelevement', 'DatePrel', 'HeurePrel', 'DateAna', 'HeureAna',
       'CdParametre', 'LbLongParamètre', 'RsAna', 'CdUniteMesure',
       'SymUniteMesure', 'CdRqAna', 'MnemoRqAna', 'CdInsituAna', 'LbInsituAna',
       'ProfondeurPrel', 'CdDifficulteAna', 'MnemoDifficulteAna', 'LdAna',
       'LqAna', 'LsAna', 'IncertAna', 'CdMetFractionnement',
       'NomMetFractionnement', 'CdMethode', 'NomMethode', 'RdtExtraction',
       'CdMethodeExtraction', 'NomMethodeExtraction', 'CdAccreAna',
       'MnemoAccredAna', 'AgreAna', 'CdStatutAna', 'MnemoStatutAna',
       'CdQualAna', 'LbQualAna', 'CommentairesAna', 'ComResultatAna', 'CdRdd',
       'NomRdd', 'CdProducteur', 'NomProducteur', 'CdPreleveur',
       'NomPreleveur', 'CdLaboratoire', 'NomLaboratoire'],
      dtype='object')

In [4]:
analyses_orthophosphate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9729 entries, 0 to 9728
Data columns (total 51 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   CdStationMesureEauxSurface  9729 non-null   int64         
 1   LbStationMesureEauxSurface  9729 non-null   object        
 2   CdSupport                   9729 non-null   int64         
 3   LbSupport                   9729 non-null   object        
 4   CdFractionAnalysee          9729 non-null   int64         
 5   LbFractionAnalysee          9729 non-null   object        
 6   CdPrelevement               9729 non-null   int64         
 7   DatePrel                    9729 non-null   datetime64[ns]
 8   HeurePrel                   9724 non-null   object        
 9   DateAna                     9710 non-null   object        
 10  HeureAna                    9524 non-null   object        
 11  CdParametre                 9729 non-null   int64       

In [5]:
analyses_orthophosphate = analyses_orthophosphate[pd.to_datetime(analyses_orthophosphate['DatePrel']).dt.year != 2022]
analyses_orthophosphate = analyses_orthophosphate[pd.to_datetime(analyses_orthophosphate['DatePrel']).dt.year != 2023]

In [6]:
new_df = analyses_orthophosphate.loc[:, ['CdStationMesureEauxSurface', 'CdPrelevement', 'RsAna', 'DatePrel']]
print(new_df)


      CdStationMesureEauxSurface  CdPrelevement  RsAna   DatePrel
0                        2000003        6994353  0.430 2020-02-03
1                        2000003        7027672  0.353 2020-05-25
2                        2000003        7031837  1.470 2020-07-20
3                        2000003        7035763  9.497 2020-09-14
4                        2000003        7037335  4.601 2020-11-09
...                          ...            ...    ...        ...
9708                     6455540         740681  0.150 2019-03-06
9725                     6455540         823738  0.430 2020-12-02
9726                     6455540         823740  0.640 2020-06-04
9727                     6455540         823929  0.190 2020-03-05
9728                     6455540         823930  0.360 2020-09-09

[9658 rows x 4 columns]


In [7]:
new_df['Month'] = pd.to_datetime(new_df['DatePrel']).dt.month
new_df['Year'] = pd.to_datetime(new_df['DatePrel']).dt.year

new_df = new_df.dropna(subset=['RsAna'])
preprocess_df = new_df.drop(columns=['CdPrelevement', 'DatePrel'])
# Remove stations with less than 15 rows


In [8]:

df = px.data.tips()
fig = px.box(preprocess_df.groupby('CdStationMesureEauxSurface').size().sort_values(ascending=False))
fig.show()
station_counts = preprocess_df.groupby('CdStationMesureEauxSurface').size().sort_values(ascending=False)
station_counts

CdStationMesureEauxSurface
2001050    78
2001700    78
2096480    40
2043750    40
2001500    40
           ..
2035750     1
2035500     1
2032800     1
2044400     1
2041850     1
Length: 395, dtype: int64

In [9]:

preprocess_df = preprocess_df.groupby('CdStationMesureEauxSurface').filter(lambda x: len(x) > 33 and len(x) <= 50)


In [10]:
station_counts = preprocess_df.groupby('CdStationMesureEauxSurface').size().sort_values(ascending=False)
station_counts


CdStationMesureEauxSurface
2095600    40
2001500    40
2043750    40
2096480    40
2094973    40
           ..
2043775    34
2048960    34
2080200    34
2084200    34
2000010    34
Length: 206, dtype: int64

In [11]:
df = px.data.tips()
fig = px.box(preprocess_df.groupby('CdStationMesureEauxSurface').size().sort_values(ascending=False))
fig.show()

In [13]:
# IDs of the I2M2 stations
station_id_check = [2001000, 2001006, 2001016, 2001025, 2001030, 2001458, 2001500,
       2001725, 2001738, 2001750, 2001945, 2001990, 2002000, 2002800,
       2003100, 2003200, 2003350, 2003397, 2003400, 2003610, 2003617,
       2003670, 2003800, 2004000, 2004300, 2004990, 2005688, 2005700,
       2006450, 2007000, 2009000, 2009085, 2010000, 2011000, 2013000,
       2015500, 2016050, 2017000, 2017500, 2017850, 2018000, 2018500,
       2018780, 2019000, 2020000, 2021000, 2022465, 2022670, 2022675,
       2023000, 2024000, 2025100, 2025200, 2025700, 2026200, 2026500,
       2028000, 2028300, 2028500, 2029000, 2029160, 2030200, 2030310,
       2030500, 2031400, 2031650, 2032000, 2035000, 2035750, 2036000,
       2037400, 2037500, 2041000, 2041100, 2041230, 2041650, 2041750,
       2041950, 2042000, 2042050, 2042555, 2043017, 2043500, 2043600,
       2043655, 2043750, 2044100, 2044400, 2045000, 2045050, 2045150,
       2045200, 2045283, 2045350, 2045425, 2045500, 2046000, 2046550,
       2047000, 2047500, 2047750, 2048960, 2048980, 2049000, 2049500,
       2049900, 2050000, 2051500, 2051600, 2051820, 2052000, 2052500,
       2054900, 2055100, 2057130, 2057168, 2057245, 2057453, 2057480,
       2061500, 2061970, 2063000, 2064770, 2065090, 2065280, 2065500,
       2068700, 2068747, 2068790, 2068800, 2070500, 2080200, 2080500,
       2081000, 2081030, 2081100, 2081300, 2081700, 2081850, 2082350,
       2082845, 2082878, 2082970, 2082985, 2084100, 2084650, 2084800,
       2089000, 2092600, 2093170, 2093600, 2094000, 2094500, 2094700,
       2094800, 2094920, 2094930, 2094940, 2094950, 2094973, 2094979,
       2095325, 2096480, 2096500, 2096520, 2096645, 2096900, 2096975,
       2096990, 2097000, 2098100, 2098200, 2098800, 2098980, 2099550,
       2099800, 2100100, 2100603, 2100830, 2101050, 2103800, 2103850,
       2103950, 2104000, 2104400, 2105000, 2106430, 2106665, 2106750,
       2106800, 2106850, 2106862, 2106900, 2107600, 6000990, 6000998,
       6001312, 6001313, 6001314, 6001316, 6091100, 6408800, 6455540]

In [19]:
preprocess_df['CdStationMesureEauxSurface'].unique()

array([2000010, 2000011, 2000990, 2001000, 2001016, 2001025, 2001030,
       2001500, 2001715, 2001725, 2001738, 2001750, 2002000, 2003100,
       2003200, 2003350, 2003400, 2003800, 2004000, 2004300, 2005700,
       2007000, 2009000, 2010000, 2011000, 2013000, 2016050, 2017000,
       2018000, 2018500, 2018780, 2019000, 2019500, 2020000, 2021000,
       2022000, 2022650, 2022700, 2022800, 2023000, 2024000, 2025500,
       2025700, 2026200, 2026250, 2026500, 2027000, 2028000, 2028500,
       2029000, 2029160, 2030200, 2030310, 2030350, 2030450, 2030500,
       2031400, 2031650, 2032000, 2035000, 2036000, 2036250, 2036265,
       2037000, 2037335, 2037350, 2037400, 2037450, 2037472, 2037500,
       2038000, 2041000, 2041230, 2041650, 2041735, 2041750, 2041950,
       2042000, 2042500, 2043350, 2043500, 2043600, 2043725, 2043750,
       2043775, 2044000, 2044300, 2045000, 2045050, 2045150, 2045200,
       2045283, 2045500, 2046000, 2046500, 2046800, 2047000, 2047500,
       2047750, 2048

In [21]:
station_ortho_check = [2000010, 2000011, 2000990, 2001000, 2001016, 2001025, 2001030,
       2001500, 2001715, 2001725, 2001738, 2001750, 2002000, 2003100,
       2003200, 2003350, 2003400, 2003800, 2004000, 2004300, 2005700,
       2007000, 2009000, 2010000, 2011000, 2013000, 2016050, 2017000,
       2018000, 2018500, 2018780, 2019000, 2019500, 2020000, 2021000,
       2022000, 2022650, 2022700, 2022800, 2023000, 2024000, 2025500,
       2025700, 2026200, 2026250, 2026500, 2027000, 2028000, 2028500,
       2029000, 2029160, 2030200, 2030310, 2030350, 2030450, 2030500,
       2031400, 2031650, 2032000, 2035000, 2036000, 2036250, 2036265,
       2037000, 2037335, 2037350, 2037400, 2037450, 2037472, 2037500,
       2038000, 2041000, 2041230, 2041650, 2041735, 2041750, 2041950,
       2042000, 2042500, 2043350, 2043500, 2043600, 2043725, 2043750,
       2043775, 2044000, 2044300, 2045000, 2045050, 2045150, 2045200,
       2045283, 2045500, 2046000, 2046500, 2046800, 2047000, 2047500,
       2047750, 2048960, 2048980, 2049000, 2049500, 2049900, 2050000,
       2051000, 2051600, 2051820, 2052000, 2052500, 2054000, 2054100,
       2054500, 2057060, 2057065, 2057100, 2057150, 2057210, 2057250,
       2057400, 2061500, 2061970, 2063000, 2064000, 2064770, 2065090,
       2065280, 2065500, 2068800, 2068900, 2070500, 2079000, 2079250,
       2079500, 2080200, 2081000, 2081100, 2081135, 2081300, 2081500,
       2081700, 2081850, 2082350, 2082710, 2082900, 2082970, 2084000,
       2084100, 2084200, 2084400, 2089000, 2089900, 2090000, 2092000,
       2093100, 2093170, 2093200, 2093600, 2094000, 2094500, 2094700,
       2094800, 2094900, 2094920, 2094930, 2094950, 2094973, 2094979,
       2095500, 2095600, 2096000, 2096400, 2096480, 2096500, 2096750,
       2096900, 2096950, 2096975, 2097000, 2098200, 2098800, 2099500,
       2099800, 2100150, 2100190, 2100603, 2100830, 2101050, 2103800,
       2103850, 2103950, 2104000, 2104400, 2104800, 2105000, 2106220,
       2106230, 2106380, 2106410, 2106430, 2106435, 2106750, 2106800,
       2106850, 2106900, 2107600]

In [23]:
# How many stations are common to the two datasets
len(set(station_id_check) & set(station_ortho_check))

131